In [ ]:
# 네이버 쇼핑 리뷰 (https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt) 데이터세트를 이용하여 다음에 답하세요.

# 1. 데이터/텍스트 전처리를 하세요.(결측치, 중복데이터, 한글 이외의 데이터 제거등)(20)

# 2. Okt를 사용하여 한글 형태소 분석을 하세요.(10)

# 3. CountVectorizer와 LogisticRegression을 이용하여 이진 분류를 하되, 최적의   파라메터를 도출하고 분류 정확도를 표시하세요.(20)

In [1]:
import numpy as np
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt'
df = pd.read_table(url, names=['score', 'review'])
df.head()

,score,review
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [3]:
# 결측치 확인
df.isna().sum()

score     0
review    0
dtype: int64

In [4]:
# 중복 데이터 확인
print(df.shape)
print(df.review.nunique())

(200000, 2)
199908


In [5]:
# 중복 데이터 제거
df.drop_duplicates(subset=['review'], inplace=True)
df.shape

(199908, 2)

In [6]:
# 한글 이외의 데이터는 제거
df.review = df.review.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()

In [7]:
df.review.replace('', np.nan, inplace=True)
df.isna().sum()

score     0
review    0
dtype: int64

In [ ]:
# 2. Okt를 사용하여 한글 형태소 분석을 하세요.(10)

In [8]:
with open('data/불용어.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]

In [9]:
from konlpy.tag import Okt
okt = Okt()

In [10]:
reviews = []
for review in df.review:
    morphs = okt.morphs(review, stem=True)
    tmp = [word for word in morphs if word not in stop_words]
    reviews.append(' '.join(tmp))

In [ ]:
# 3. CountVectorizer와 LogisticRegression을 이용하여 이진 분류를 하되, 최적의   파라메터를 도출하고 분류 정확도를 표시하세요.(20)

In [11]:
# 평점이 4, 5 점은 긍정(1), 나머지는 부정(0)
df.score = df.score.apply(lambda x: 1 if x >= 4 else 0)
df.head()

,score,review
0,1,배공빠르고 굿
1,0,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,1,아주좋아요 바지 정말 좋아서 개 더 구매했어요 이가격에 대박입니다 바느질이 조금 ...
3,0,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전...
4,1,민트색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [12]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(
    reviews, df.score.values, stratify=df.score.values, random_state=2023
)

In [13]:
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.linear_model import LogisticRegression 

In [14]:
cvect = CountVectorizer()
lrc = LogisticRegression(random_state=2023)

params = { 
    'CVECT__ngram_range': [(1,1), (1,2)],
    'LRC__C': [1,10]
}

pipeline = Pipeline([ ('CVECT', cvect),('LRC', lrc)])
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
grid_pipe.fit(X_train, y_train)
grid_pipe.best_params_

c:\Users\YONSAI\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\YONSAI\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

{'CVECT__ngram_range': (1, 2), 'LRC__C': 1}

In [15]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.8970126258078717